## M2 - Model prediksi titik panas di Sumatera Selatan 2001 - 2023

In [ ]:
# load all functions
from C01_data_collection import *
from C02_visualization import *
from C03_preprocessing import *
from C04_model_predictions import *
from C05_model_evaluate import *

- config models

In [ ]:
# # set random number
# import random as rm
# rm.seed(1234)

# # set random number
# import numpy as np
# np.random.seed(1234)

# set random number
import tensorflow as tf
tf.random.set_seed(1234)

### 1. Data Collections

In [ ]:
# load dataset
dataset = data_hotspot("dataset_enso.csv")
print(dataset.tail(3))

### 2. Data Preprocessing

In [ ]:
# prosess normalization min-max
scaler, scaled = normalized(dataset, ["hotspot", "sst_anom", "oni_anom", "soi_anom"])

In [ ]:
# hasil normalisasi data
np.round(scaled[:3],5)

### 3. Pembagian Data

In [ ]:
# proses pembagian data
x_train, y_train, x_test, y_test = results_multivariate_supervised(scaled)

In [ ]:
# cek dimensi data
print(x_train.shape, y_train.shape)

In [ ]:
# cek dimensi data
print(x_test.shape, y_test.shape)

### 4. Algoritma LSTM-RNN

#### 4.1 SBi-LSTM-RNN

In [ ]:
# call models LSTM
# param : algorithm, timestep, activation, optimizer, dropout
lstm_model = get_models(
  algorithm="SBi-LSTM", timestep=x_train, activation="relu", optimizer="rmsprop", dropout=0.20
)

In [ ]:
# compile models
# param : model, batch_size, epochs, x_train, y_train, x_test, y_test
lstm_history, lstm_predictions = get_prediction(
  model=lstm_model, batch_size=32, epochs=1500, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test
)

In [ ]:
# show loss function
lineplot2(
  x1=lstm_history.epoch, y1=lstm_history.history['loss'], label1="Training",
  x2=lstm_history.epoch, y2=lstm_history.history['val_loss'], label2="Validation",
  title="Loss Function of SBi-LSTM"
)

In [ ]:
# show result predictions
scaler,_ = normalized(dataset, ["hotspot"])
lineplot3(
  x1=dataset["acq_date"].iloc[216:-1], y1=inverse(scaler, y_test.reshape(-1,1)), label1="actual data",
  x2=dataset["acq_date"].iloc[216:-1], y2=inverse(scaler, lstm_predictions.reshape(-1,1)), label2="results predictions",
  title="Results of Predictions Hotspot with SBi-LSTM"
)

In [ ]:
# process evaluate models
r, p, mae, rmse, mape = evaluate_models(
  ytrue=y_test,
  ypred=lstm_predictions,
)

# results evaluate models
print("Evaluate Models with SBi-LSTM")
print("-----------------------------")
print("R       : "+str(r))
print("P-value : "+str(p))
print("MAE     : "+str(mae))
print("RMSE    : "+str(rmse))
print("MAPE    : "+str(mape))

In [ ]:
# process evaluate models
r, p, mae, rmse, mape = evaluate_models(
  ytrue=inverse(scaler, y_test.reshape(-1,1)),
  ypred=inverse(scaler, lstm_predictions.reshape(-1,1)),
)

# results evaluate models
print("Evaluate Models with SBi-LSTM")
print("-----------------------------")
print("R       : "+str(r))
print("P-value : "+str(p))
print("MAE     : "+str(mae))
print("RMSE    : "+str(rmse))
print("MAPE    : "+str(mape*100))

#### 4.2 SBi-LSTM-XGBoost

In [ ]:
# call models XGBoost
lstm_xgb = get_XGBoost(
  xtrue=x_test, ytrue=y_test, ypred=lstm_predictions
)

In [ ]:
# show result predictions
scaler,_ = normalized(dataset, ["hotspot"])
lineplot3(
  x1=dataset["acq_date"].iloc[216:-1], y1=inverse(scaler, y_test.reshape(-1,1)), label1="actual data",
  x2=dataset["acq_date"].iloc[216:-1], y2=inverse(scaler, lstm_xgb.reshape(-1,1)), label2="results predictions",
  title="Results of Predictions Hotspot with SBi-LSTM-XGBoost"
)

In [ ]:
# process evaluate models
r, p, mae, rmse, mape = evaluate_models(
  ytrue=y_test,
  ypred=lstm_xgb,
)

# results evaluate models
print("Evaluate Models with SBi-LSTM-XGBoost")
print("-------------------------------------")
print("R       : "+str(r))
print("P-value : "+str(p))
print("MAE     : "+str(mae))
print("RMSE    : "+str(rmse))
print("MAPE    : "+str(mape))

In [ ]:
# process evaluate models
r, p, mae, rmse, mape = evaluate_models(
  ytrue=inverse(scaler, y_test.reshape(-1,1)),
  ypred=inverse(scaler, lstm_xgb.reshape(-1,1)),
)

# results evaluate models
print("Evaluate Models with SBi-LSTM-XGBoost")
print("-------------------------------------")
print("R       : "+str(r))
print("P-value : "+str(p))
print("MAE     : "+str(mae))
print("RMSE    : "+str(rmse))
print("MAPE    : "+str(mape))

In [ ]:
# show result predictions
lineplot4(

  # sumbu x
  date=dataset["acq_date"].iloc[216:-1],
  
  # data actual
  ytrue=inverse(scaler, y_test.reshape(-1,1)),

  # hasil SBi-LSTM
  ypred1=inverse(scaler, lstm_predictions.reshape(-1,1)),
  label1="results SBi-LSTM",
  
  # hasil SBi-LSTM-XGBoost
  ypred2=inverse(scaler, lstm_xgb.reshape(-1,1)),
  label2="results SBi-LSTM-XGBoost",

  # title plot
  title="Results Prediction of Hotspot with SBi-LSTM-XGBoost"
)

### 5. Algoritma GRU-RNN

In [ ]:
# set random number
import tensorflow as tf
tf.random.set_seed(42)

#### 5.1 SBi-GRU-RNN

In [ ]:
# call models GRU
# param : algorithm, timestep, activation, optimizer, dropout
gru_model = get_models(
  algorithm="SBi-GRU", timestep=x_train, activation="relu", optimizer="rmsprop", dropout=0.20
)

In [ ]:
# compile models
# param : model, batch_size, epochs, x_train, y_train, x_test, y_test
gru_history, gru_predictions = get_prediction(
  model=gru_model, batch_size=16, epochs=1500, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test
)

In [ ]:
# show loss function
lineplot2(
  x1=gru_history.epoch, y1=gru_history.history['loss'], label1="Training",
  x2=gru_history.epoch, y2=gru_history.history['val_loss'], label2="Validation",
  title="Loss Function of SBi-GRU"
)


In [ ]:
# show result predictions
scaler,_ = normalized(dataset, ["hotspot"])
lineplot3(
  x1=dataset["acq_date"].iloc[216:-1], y1=inverse(scaler, y_test.reshape(-1,1)), label1="actual data",
  x2=dataset["acq_date"].iloc[216:-1], y2=inverse(scaler, gru_predictions.reshape(-1,1)), label2="results predictions",
  title="Results of Predictions Hotspot with SBi-GRU"
)

In [ ]:
# process evaluate models
r, p, mae, rmse, mape = evaluate_models(
  ytrue=y_test,
  ypred=gru_predictions,
)

# results evaluate models
print("Evaluate Models with SBi-GRU")
print("-----------------------------")
print("R       : "+str(r))
print("P-value : "+str(p))
print("MAE     : "+str(mae))
print("RMSE    : "+str(rmse))
print("MAPE    : "+str(mape))


In [ ]:
# process evaluate models
r, p, mae, rmse, mape = evaluate_models(
  ytrue=inverse(scaler, y_test.reshape(-1,1)),
  ypred=inverse(scaler, gru_predictions.reshape(-1,1)),
)

# results evaluate models
print("Evaluate Models with SBi-GRU")
print("-----------------------------")
print("R       : "+str(r))
print("P-value : "+str(p))
print("MAE     : "+str(mae))
print("RMSE    : "+str(rmse))
print("MAPE    : "+str(mape*100))

#### 5.2 SBi-GRU-XGBoost

In [ ]:
# call models XGBoost
gru_xgb = get_XGBoost(
  xtrue=x_test, ytrue=y_test, ypred=gru_predictions
)

In [ ]:
# show result predictions
scaler,_ = normalized(dataset, ["hotspot"])
lineplot3(
  x1=dataset["acq_date"].iloc[216:-1], y1=inverse(scaler, y_test.reshape(-1,1)), label1="actual data",
  x2=dataset["acq_date"].iloc[216:-1], y2=inverse(scaler, gru_xgb.reshape(-1,1)), label2="results predictions",
  title="Results of Predictions Hotspot with SBi-GRU-XGBoost"
)

In [ ]:
# process evaluate models
r, p, mae, rmse, mape = evaluate_models(
  ytrue=y_test,
  ypred=gru_xgb,
)

# results evaluate models
print("Evaluate Models with SBi-GRU-XGBoost")
print("-------------------------------------")
print("R       : "+str(r))
print("P-value : "+str(p))
print("MAE     : "+str(mae))
print("RMSE    : "+str(rmse))
print("MAPE    : "+str(mape))

In [ ]:
# process evaluate models
r, p, mae, rmse, mape = evaluate_models(
  ytrue=inverse(scaler, y_test.reshape(-1,1)),
  ypred=inverse(scaler, gru_xgb.reshape(-1,1)),
)

# results evaluate models
print("Evaluate Models with SBi-GRU-XGBoost")
print("-------------------------------------")
print("R       : "+str(r))
print("P-value : "+str(p))
print("MAE     : "+str(mae))
print("RMSE    : "+str(rmse))
print("MAPE    : "+str(mape))

In [ ]:
# show result predictions
lineplot4(

  # sumbu x
  date=dataset["acq_date"].iloc[216:-1],
  
  # data actual
  ytrue=inverse(scaler, y_test.reshape(-1,1)),

  # hasil SBi-GRU
  ypred1=inverse(scaler, gru_predictions.reshape(-1,1)),
  label1="results SBi-GRU",
  
  # hasil SBi-GRU-XGBoost
  ypred2=inverse(scaler, gru_xgb.reshape(-1,1)),
  label2="results SBi-GRU-XGBoost",

  # title plot
  title="Results Prediction of Hotspot with SBi-GRU-XGBoost"
)

### 6. Results Predictions Hotspot

In [ ]:
# Results predictions
pd.concat([
  pd.DataFrame(np.array(dataset["acq_date"].iloc[217:]), columns=["Date"]),
  pd.DataFrame(np.round(inverse(scaler, y_test.reshape(-1,1)),0), columns=["Ytrue"]),
  pd.DataFrame(np.round(inverse(scaler, lstm_predictions.reshape(-1,1)),0), columns=["SBi-LSTM"]),
  pd.DataFrame(np.round(inverse(scaler, lstm_xgb.reshape(-1,1)),0), columns=["SBi-LSTM-XGBoost"]),
  pd.DataFrame(np.round(inverse(scaler, gru_predictions.reshape(-1,1)),0), columns=["SBi-GRU"]),
  pd.DataFrame(np.round(inverse(scaler, gru_xgb.reshape(-1,1)),0), columns=["SBi-GRU-XGBoost"]),
], axis=1).tail()
# 
# 
# #.to_csv("hasil prediksi titik panas M2.csv", index=False)